In [ ]:
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_script import Manager
from flask_migrate import Migrate, MigrateCommand
from flask_api import FlaskAPI
from sqlalchemy_utils import register_composites
from simplejson import dumps

app = FlaskAPI(__name__)
app.config.from_object(os.environ['APP_SETTINGS'])
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

db = SQLAlchemy(app)
migrate = Migrate(app, db)

conn = db.session.connection()
register_composites(conn)

manager = Manager(app)
manager.add_command('db', MigrateCommand)

from snomedct_terminology_server.models import models

#####
# routes
#####

@app.route('/concepts/<int:id>')
def get_concept(id):
    concept_row_record = models.CurrentSnapshotDenormalizedConcept.query.filter_by(id=id)
    try:
        concept = concept_row_record.first_or_404()
        return concept._to_dict()
    except Exception as e:
        return e

@app.route('/refsets/<int:referenced_component_id>')
def get_refset(referenced_component_id):
    refset_row_record = models.CurrentSnapshotDenormalizedRefset.query.filter_by(referenced_component_id=referenced_component_id)
    try:
        refset = refset_row_record.first_or_404()
        return refset._to_dict()
    except Exception as e:
        return e


@app.route('/relationships/<int:id>')
def get_relationship(id):
    relationship_row_record = models.CurrentSnapshotDenormalizedRelationship.query.filter_by(id=id)
    try:
        relationship = relationship_row_record.first_or_404()
        return relationship._to_dict()
    except Exception as e:
        return e

@app.route('/description/concept/<int:concept_id>')
def get_description_for_concept(concept_id):
    description_row_records = models.CurrentSnapshotDenormalizedDescription.query.filter_by(concept_id=concept_id)
    try:
        descriptions = description_row_records.all()
        return list(map(lambda elt: elt._to_dict(), descriptions))
    except Exception as e:
        return e

@app.route('/description/<int:id>')
def get_description(id):
    description_row_record = models.CurrentSnapshotDenormalizedDescription.query.filter_by(id=id)
    try:
        description = description_row_record.first_or_404()
        return description._to_dict()
    except Exception as e:
        return e


@app.route('/refset_memberships/<int:referenced_component_id>')
def get_refset_memberships(referenced_component_id):
    refset_memberships_records = models.CurrentSnapshotDenormalizedRefset.query.filter_by(referenced_component_id=referenced_component_id)
    try:
        register_composites(conn)
        refset_memberships = refset_memberships_records.all()
        return list(map(lambda elt: elt._to_dict(), refset_memberships))
    except Exception as e:
        return str(e)

@app.route('/language_refset_memberships/<int:referenced_component_id>')
def get_language_refset_memberships(referenced_component_id):
    language_refset_memberships_records = models.CurrentSnapshotDenormalizedLanguageRefset.query.filter_by(referenced_component_id=referenced_component_id)
    try:
        language_refset_memberships = language_refset_memberships_records.all()
        return list(map(lambda elt: elt._to_dict(), language_refset_memberships))
    except Exception as e:
        return str(e)


if __name__ == '__main__':
    app.run()


In [ ]:
from sqlalchemy.dialects.postgresql import JSON
from snomedct_terminology_server.app import db

from snomedct_terminology_server.data_types.custom_types import UUID
from snomedct_terminology_server.mixins.dict_mixin import DictTransform

from sqlalchemy_utils import CompositeArray
from sqlalchemy_utils import CompositeType

class DenormalizedComponent(DictTransform, db.Model):
    __abstract__ = True

    effective_time = db.Column(db.Date, primary_key=True)
    active = db.Column(db.Boolean, primary_key=True)
    module_id = db.Column(db.BigInteger, primary_key=True, autoincrement=False)


class TransitiveClosure(DictTransform, db.Model):
    __tablename__ = 'transitive_closure_for_current_snapshot'

    active = db.Column(db.Boolean, primary_key=True)
    effective_time = db.Column(db.Date, primary_key=True)
    supertype_id = db.Column(db.BigInteger, primary_key=True, autoincrement=False)
    subtype_id = db.Column(db.BigInteger, primary_key=True, autoincrement=False)

    __table_args__ = (
        db.Index('ix_%s_inv' % __tablename__,
                 'supertype_id',
                 postgresql_using='btree'
        ),
        db.Index('ix_%s_main' % __tablename__,
                 'subtype_id', 'supertype_id',
                 postgresql_using='btree'
        ),
    )


class CurrentSnapshotDenormalizedConcept(DenormalizedComponent):
    __tablename__ = 'snomed_denormalized_concept_view_for_current_snapshot'

    id = db.Column(db.BigInteger, primary_key=True, index=True, autoincrement=False)
    module_name = db.Column(db.Text)
    definition_status_id = db.Column(db.BigInteger)
    definition_status_name = db.Column(db.Text)
    is_primitive = db.Column(db.Boolean)
    fully_specified_name = db.Column(db.Text)
    preferred_term = db.Column(db.Text)
    definition = db.Column(JSON)
    descriptions = db.Column(JSON)
    parents = db.Column(JSON)
    children = db.Column(JSON)
    ancestors = db.Column(JSON)
    descendants = db.Column(JSON)
    incoming_relationships = db.Column(JSON)
    outgoing_relationships = db.Column(JSON)
    reference_set_memberships = db.Column(JSON)


class CurrentSnapshotDenormalizedRelationship(DenormalizedComponent):
    __tablename__ = 'denormalized_relationship_for_current_snapshot'

    id = db.Column(db.BigInteger, primary_key=True, index=True, autoincrement=False)
    module_name = db.Column(db.Text)
    relationship_group = db.Column(db.Integer)
    source_id = db.Column(db.BigInteger)
    source_name = db.Column(db.Text)
    destination_id = db.Column(db.BigInteger)
    destination_name = db.Column(db.Text)
    type_id = db.Column(db.BigInteger)
    type_name = db.Column(db.Text)
    characteristic_type_id = db.Column(db.BigInteger)
    characteristic_type_name = db.Column(db.Text)
    modifier_id = db.Column(db.BigInteger)
    modifier_name = db.Column(db.Text)

    __table_args__ = (
        db.Index('sct_%s_type_id' % __tablename__,
                 'type_id',
                 postgresql_using='btree',
                 postgresql_where= (type_id == 116680003)
        ),)


class CurrentSnapshotDenormalizedDescription(DenormalizedComponent):
    __tablename__ = 'denormalized_description_for_current_snapshot'

    id = db.Column(db.BigInteger, primary_key=True, index=True, autoincrement=False)
    module_name = db.Column(db.Text)
    language_code = db.Column(db.Text)
    type_id = db.Column(db.BigInteger, index=True)
    type_name = db.Column(db.Text)
    term = db.Column(db.Text)
    case_significance_id = db.Column(db.BigInteger)
    case_significance_name = db.Column(db.Text)
    concept_id = db.Column(db.BigInteger, index=True)
    language_reference_set_memberships = db.Column(JSON)

    __table_args__ = (
        db.Index('fsn_%s_type_id' % __tablename__,
                 'type_id',
                 postgresql_using='btree',
                 postgresql_where= (type_id == 900000000000003001)
        ),)


class CurrentSnapshotDenormalizedLanguageRefset(DenormalizedComponent):
    __tablename__ = 'denormalized_language_reference_set'

    uuid = db.Column(UUID(binary=False), primary_key=True)
    refset = db.Column(JSON)
    referenced_component_id = db.Column(db.BigInteger, index=True)


class CurrentSnapshotDenormalizedRefset(DenormalizedComponent):
    __tablename__ = 'denormalized_reference_set'

    uuid = db.Column(UUID(binary=False), primary_key=True)
    refset = db.Column(CompositeArray(
        CompositeType(
            'denormalized_refset_concept',
            [
                db.Column('concept_id', db.BigInteger, nullable=False),
                db.Column('preferred_term', db.Text),
                db.Column('definition_status', db.Text),
                db.Column('active', db.Boolean),
                db.Column('effective_time', db.Date),
                db.Column('module', db.BigInteger, nullable=False)
            ]
        )
    ))

    referenced_component_id = db.Column(db.BigInteger, index=True)


In [ ]:
import datetime
import uuid
import sqlalchemy_utils

class DictTransform(object):
    def _to_dict(self):
        fields = {}
        for field in [x for x in dir(self) if not x.startswith('_') and x != 'metadata' and not x.startswith('query')]:
            val = self.__getattribute__(field)
            if isinstance(val, datetime.date):
                val = val.isoformat()
            elif isinstance(val, uuid.UUID):
                val = str(val)
            elif isinstance(val, list) and len(val) > 0 and 'definition_status' in dir(val[0]):
                denormalized_refset_concept = val[0]
                val = {'concept_id': denormalized_refset_concept.concept_id,
                       'preferred_term': denormalized_refset_concept.preferred_term,
                       'definition_status': denormalized_refset_concept.definition_status,
                       'active':denormalized_refset_concept.active,
                       'effective_time': str(denormalized_refset_concept.effective_time),
                       'module': denormalized_refset_concept.module}
            fields[field] = val

        return fields


In [ ]:
import sqlalchemy as sa
from sqlalchemy_utils.types.uuid import UUIDType

class UUID(UUIDType):
  def __repr__(self):
    return "sa.dialects.postgresql.UUID()"


In [ ]:
from alembic import op
import sqlalchemy as sa
from sqlalchemy.sql import text
import sqlalchemy_utils
import snomedct_terminology_server

from sqlalchemy.dialects import postgresql

def upgrade():
    ### commands auto generated by Alembic - please adjust! ###
    op.get_bind().execute(text("""
CREATE TYPE denormalized_refset_concept AS (
    concept_id bigint,
    preferred_term text,
    definition_status text,
    active boolean,
    effective_time date,
    module bigint
)
    """))
    op.create_table('denormalized_description_for_current_snapshot',
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('module_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('module_name', sa.Text(), nullable=True),
    sa.Column('language_code', sa.Text(), nullable=True),
    sa.Column('type_id', sa.BigInteger(), nullable=True),
    sa.Column('type_name', sa.Text(), nullable=True),
    sa.Column('term', sa.Text(), nullable=True),
    sa.Column('case_significance_id', sa.BigInteger(), nullable=True),
    sa.Column('case_significance_name', sa.Text(), nullable=True),
    sa.Column('concept_id', sa.BigInteger(), nullable=True),
    sa.Column('language_reference_set_memberships', postgresql.JSON(), nullable=True),
    sa.PrimaryKeyConstraint('effective_time', 'active', 'module_id', 'id')
    )
    op.create_index('fsn_denormalized_description_for_current_snapshot_type_id', 'denormalized_description_for_current_snapshot', ['type_id'], unique=False, postgresql_using='btree', postgresql_where=sa.text('type_id = 900000000000003001'))
    op.create_index(op.f('ix_denormalized_description_for_current_snapshot_concept_id'), 'denormalized_description_for_current_snapshot', ['concept_id'], unique=False)
    op.create_index(op.f('ix_denormalized_description_for_current_snapshot_id'), 'denormalized_description_for_current_snapshot', ['id'], unique=False)
    op.create_index(op.f('ix_denormalized_description_for_current_snapshot_type_id'), 'denormalized_description_for_current_snapshot', ['type_id'], unique=False)
    op.create_table('denormalized_language_reference_set',
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('module_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('uuid', snomedct_terminology_server.data_types.custom_types.sa.dialects.postgresql.UUID(), nullable=False),
    sa.Column('refset', postgresql.JSON(), nullable=True),
    sa.Column('referenced_component_id', sa.BigInteger(), nullable=True),
    sa.PrimaryKeyConstraint('effective_time', 'active', 'module_id', 'uuid')
    )
    op.create_index(op.f('ix_denormalized_language_reference_set_referenced_component_id'), 'denormalized_language_reference_set', ['referenced_component_id'], unique=False)
    op.create_table('denormalized_reference_set',
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('module_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('uuid', snomedct_terminology_server.data_types.custom_types.sa.dialects.postgresql.UUID(), nullable=False),
    sa.Column('refset', sqlalchemy_utils.types.pg_composite.CompositeArray(sqlalchemy_utils.types.pg_composite.CompositeType('denormalized_refset_concept', [sa.Column('concept_id', sa.BigInteger(), nullable=False), sa.Column('preferred_term', sa.Text()), sa.Column('definition_status', sa.Text()), sa.Column('active', sa.Boolean()), sa.Column('effective_time', sa.Date()), sa.Column('module', sa.BigInteger(), nullable=False)])), nullable=True),
    sa.Column('referenced_component_id', sa.BigInteger(), nullable=True),
    sa.PrimaryKeyConstraint('effective_time', 'active', 'module_id', 'uuid')
    )
    op.create_index(op.f('ix_denormalized_reference_set_referenced_component_id'), 'denormalized_reference_set', ['referenced_component_id'], unique=False)
    op.create_table('denormalized_relationship_for_current_snapshot',
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('module_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('module_name', sa.Text(), nullable=True),
    sa.Column('relationship_group', sa.Integer(), nullable=True),
    sa.Column('source_id', sa.BigInteger(), nullable=True),
    sa.Column('source_name', sa.Text(), nullable=True),
    sa.Column('destination_id', sa.BigInteger(), nullable=True),
    sa.Column('destination_name', sa.Text(), nullable=True),
    sa.Column('type_id', sa.BigInteger(), nullable=True),
    sa.Column('type_name', sa.Text(), nullable=True),
    sa.Column('characteristic_type_id', sa.BigInteger(), nullable=True),
    sa.Column('characteristic_type_name', sa.Text(), nullable=True),
    sa.Column('modifier_id', sa.BigInteger(), nullable=True),
    sa.Column('modifier_name', sa.Text(), nullable=True),
    sa.PrimaryKeyConstraint('effective_time', 'active', 'module_id', 'id')
    )
    op.create_index(op.f('ix_denormalized_relationship_for_current_snapshot_id'), 'denormalized_relationship_for_current_snapshot', ['id'], unique=False)
    op.create_index('sct_denormalized_relationship_for_current_snapshot_type_id', 'denormalized_relationship_for_current_snapshot', ['type_id'], unique=False, postgresql_using='btree', postgresql_where=sa.text('type_id = 116680003'))
    op.create_table('snomed_denormalized_concept_view_for_current_snapshot',
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('module_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('module_name', sa.Text(), nullable=True),
    sa.Column('definition_status_id', sa.BigInteger(), nullable=True),
    sa.Column('definition_status_name', sa.Text(), nullable=True),
    sa.Column('is_primitive', sa.Boolean(), nullable=True),
    sa.Column('fully_specified_name', sa.Text(), nullable=True),
    sa.Column('preferred_term', sa.Text(), nullable=True),
    sa.Column('definition', postgresql.JSON(), nullable=True),
    sa.Column('descriptions', postgresql.JSON(), nullable=True),
    sa.Column('parents', postgresql.JSON(), nullable=True),
    sa.Column('children', postgresql.JSON(), nullable=True),
    sa.Column('ancestors', postgresql.JSON(), nullable=True),
    sa.Column('descendants', postgresql.JSON(), nullable=True),
    sa.Column('incoming_relationships', postgresql.JSON(), nullable=True),
    sa.Column('outgoing_relationships', postgresql.JSON(), nullable=True),
    sa.Column('reference_set_memberships', postgresql.JSON(), nullable=True),
    sa.PrimaryKeyConstraint('effective_time', 'active', 'module_id', 'id')
    )
    op.create_index(op.f('ix_snomed_denormalized_concept_view_for_current_snapshot_id'), 'snomed_denormalized_concept_view_for_current_snapshot', ['id'], unique=False)
    op.create_table('transitive_closure_for_current_snapshot',
    sa.Column('active', sa.Boolean(), nullable=False),
    sa.Column('effective_time', sa.Date(), nullable=False),
    sa.Column('supertype_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.Column('subtype_id', sa.BigInteger(), autoincrement=False, nullable=False),
    sa.PrimaryKeyConstraint('active', 'effective_time', 'supertype_id', 'subtype_id')
    )
    op.create_index('ix_transitive_closure_for_current_snapshot_inv', 'transitive_closure_for_current_snapshot', ['supertype_id'], unique=False, postgresql_using='btree')
    op.create_index('ix_transitive_closure_for_current_snapshot_main', 'transitive_closure_for_current_snapshot', ['subtype_id', 'supertype_id'], unique=False, postgresql_using='btree')
    ### end Alembic commands ###


def downgrade():
    ### commands auto generated by Alembic - please adjust! ###
    op.drop_index('ix_transitive_closure_for_current_snapshot_main', table_name='transitive_closure_for_current_snapshot')
    op.drop_index('ix_transitive_closure_for_current_snapshot_inv', table_name='transitive_closure_for_current_snapshot')
    op.drop_table('transitive_closure_for_current_snapshot')
    op.drop_index(op.f('ix_snomed_denormalized_concept_view_for_current_snapshot_id'), table_name='snomed_denormalized_concept_view_for_current_snapshot')
    op.drop_table('snomed_denormalized_concept_view_for_current_snapshot')
    op.drop_index('sct_denormalized_relationship_for_current_snapshot_type_id', table_name='denormalized_relationship_for_current_snapshot')
    op.drop_index(op.f('ix_denormalized_relationship_for_current_snapshot_id'), table_name='denormalized_relationship_for_current_snapshot')
    op.drop_table('denormalized_relationship_for_current_snapshot')
    op.drop_index(op.f('ix_denormalized_reference_set_referenced_component_id'), table_name='denormalized_reference_set')
    op.drop_table('denormalized_reference_set')
    op.drop_index(op.f('ix_denormalized_language_reference_set_referenced_component_id'), table_name='denormalized_language_reference_set')
    op.drop_table('denormalized_language_reference_set')
    op.drop_index(op.f('ix_denormalized_description_for_current_snapshot_type_id'), table_name='denormalized_description_for_current_snapshot')
    op.drop_index(op.f('ix_denormalized_description_for_current_snapshot_id'), table_name='denormalized_description_for_current_snapshot')
    op.drop_index(op.f('ix_denormalized_description_for_current_snapshot_concept_id'), table_name='denormalized_description_for_current_snapshot')
    op.drop_index('fsn_denormalized_description_for_current_snapshot_type_id', table_name='denormalized_description_for_current_snapshot')
    op.drop_table('denormalized_description_for_current_snapshot')
    op.get_bind().execute(text("""
    DROP TYPE denormalized_refset_concept
    """))

    ### end Alembic commands ###


In [ ]:
import os
basedir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

class Config(object):
    DEBUG = False
    TESTING = False
    CSRF_ENABLED = True
    SECRET_KEY = os.environ['SECRET_KEY']
    SQLALCHEMY_DATABASE_URI = os.environ['DATABASE_URL']
    MIGRATIONS_PATH = os.environ['MIGRATIONS_PATH']

class ProductionConfig(Config):
    DEBUG = False


class StagingConfig(Config):
    DEVELOPMENT = True
    DEBUG = True


class DevelopmentConfig(Config):
    DEVELOPMENT = True
    DEBUG = True


class TestingConfig(Config):
    TESTING = True
